<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Load" data-toc-modified-id="Load-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#LGBM-training" data-toc-modified-id="LGBM-training-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>LGBM training</a></span><ul class="toc-item"><li><span><a href="#Prepare" data-toc-modified-id="Prepare-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Prepare</a></span></li><li><span><a href="#Core" data-toc-modified-id="Core-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Core</a></span></li></ul></li></ul></div>

# Load

In [1]:
from apps.lib import Sensor, Performance, Model
from apps.score import ScoreBoard
import apps.tkconfig as tkconfig
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from tqdm import tqdm
import lightgbm as lgb

SONG = 1
WHO = 7
TRAIN_ORDER = 3

Using TensorFlow backend.


In [2]:
sensor_data = Sensor(verbose=0)

In [3]:
scoreboard = ScoreBoard(sensor_data.drummer_df)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
def gogo(who_id, song_id, train_pf_order):
    train_pf = Performance(sensor_data, who_id, song_id, train_pf_order, 2, 2)
    
    pfs = []
    for pf_order in tqdm(range(1, 3 + 1)):
        if pf_order == train_pf_order:
            pfs.append(train_pf)
        else:
            pfs.append(Performance(sensor_data, who_id, song_id, pf_order, 2, 2)) 
            
    return pfs

In [5]:
pfs = []
for drumer in range(1, 8 + 1):
    ppf = gogo(drumer, SONG, TRAIN_ORDER)
    for j in range(3):
        ppf[j].scale()
    pfs.append(ppf)

100%|██████████| 3/3 [00:18<00:00,  6.04s/it]


In [6]:
def soso(who_id, song_id):
    sfs = []
    for pf_order in range(1, 3 + 1):
        img_scores, _ = scoreboard.get_scores_with_timestamps(who_id, song_id, pf_order)
        sfs.append(img_scores[-1])
    return sfs

In [7]:
sfs = []
for drumer in range(1, 8 + 1):
    sfs.append(soso(drumer, SONG))

3776/3900 [============================>.] - ETA: 0s

In [8]:
pfs[0][2].event_primitive_df

,hit_type,L2,L1,R1,R2,L_AAI,L_AVI,L_ASMA,L_GAI,L_GVI,...,R_AZCR,R_GZCR,R_AMCR,R_GMCR,R_AXYCORR,R_AYZCORR,R_AZXCORR,R_GXYCORR,R_GYZCORR,R_GZXCORR
0,1,0.0,0.0,0.0,0.0,-0.136028,-0.451325,-0.266619,-0.560972,-0.556726,...,-0.180426,0.307963,-0.787851,0.645677,-0.881119,-1.615020,0.162458,1.344378,0.496702,-0.041998
1,0,1.0,0.0,0.0,0.0,-0.166216,-0.431902,-0.255999,0.003551,-0.257159,...,-1.584548,-0.036693,-1.225330,0.333814,0.780290,-0.013716,-0.578081,0.962513,1.223531,0.206755
2,0,0.0,1.0,0.0,1.0,-0.231866,-0.472441,-0.232626,0.491090,-0.509810,...,-1.070845,-1.070661,-0.720547,-0.601773,0.955974,0.146481,-1.125927,1.113165,-0.361477,-1.375455
3,0,0.0,0.0,1.0,0.0,0.894816,1.372150,0.951029,0.659593,-0.256995,...,-0.300290,-0.295185,0.036629,0.099917,-1.973363,1.401021,-1.520546,-0.867884,-0.285379,0.870675
4,1,0.0,0.0,0.0,0.0,1.032245,0.948284,1.106868,0.430733,-0.218102,...,-0.180426,-1.139592,-0.316719,-0.664146,1.014875,-1.327340,-0.388272,-0.947232,1.016924,-0.085681
5,0,1.0,0.0,0.0,0.0,0.073581,-0.359904,0.137056,-0.003822,-0.496021,...,-0.180426,-1.139592,0.154412,-0.664146,1.248826,-0.791718,-1.229109,1.135236,-0.776991,-0.559269
6,0,0.0,1.0,0.0,1.0,-0.342892,-0.498899,-0.296361,-0.340387,-0.494578,...,0.114624,-0.991125,-1.186501,-1.033583,-0.226688,1.202864,-1.221966,0.338213,-0.837649,1.066046
7,0,0.0,0.0,1.0,0.0,-0.419219,-0.510526,-0.418535,-0.523127,-0.617494,...,-0.991814,0.122379,-0.642887,-0.026027,-0.334557,1.672901,-0.260754,-0.001407,-1.177179,1.211984
8,1,0.0,0.0,0.0,1.0,-0.247315,-0.481500,-0.252620,-0.520171,-0.620283,...,-1.139339,0.307963,0.154412,-0.664146,-0.798074,-0.791541,0.690142,-0.095686,-0.765724,0.319950
9,0,1.0,0.0,1.0,0.0,0.543447,0.127323,0.534156,0.280760,0.492763,...,-0.180426,-0.657074,-1.258983,-1.100754,-0.883704,-1.184597,1.153750,-1.346605,-0.060706,0.755559


# LGBM training

## Prepare

In [9]:
def prepare(train_who, train_porder, valid_who, valid_porder):
    train_df = pfs[train_who - 1][train_porder - 1].event_primitive_df
    x = train_df.drop(['hit_type'], axis=1)
    y = train_df['hit_type']
    
    valid_df = pfs[valid_who - 1][valid_porder - 1].event_primitive_df
    x_valid = valid_df.drop(['hit_type'], axis=1)
    y_valid = valid_df['hit_type']
    
    return x, y, x_valid, y_valid

In [10]:
params = dict({
    'learning_rate': 0.5,
    'application': 'multiclass',
    'num_classes': 3,
#     'min_data_in_leaf': 5,
#     'max_depth': 8,
    'num_leaves': 2 ** 3,
    'verbosity': 0,
    'metric': 'multi_error'
})

grid_params = {
    'learning_rate': [0.1, 0.2],
    'max_depth': [8, 10],
}

In [11]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    return cm

In [12]:
def get_multi_error(y, pred_y):
    cnf_matrix = confusion_matrix(y, pred_y)
    
    # Plot normalized confusion matrix
#     plt.figure(figsize=(10, 10))
    cm = plot_confusion_matrix(cnf_matrix, classes=[0, 1, 2], normalize=True,
                          title='Normalized confusion matrix')
    return sum([1 - cm[i][i] for i in range(cm.shape[0])]) / cm.shape[0]

## Core

In [13]:
result_df = pd.DataFrame(columns=['train_who', 'train_porder', 'train_score', 'valid_who', 'valid_porder', 'valid_score', 'test_who', 'test_porder', 'test_score', 'multi_error'])
id_ = 0

for train_who in range(1, 8 + 1):
    for train_porder in range(1, 3 + 1):
        for valid_who in range(1, 8 + 1):
            for valid_porder in range(1, 3 + 1):
                x, y, x_valid, y_valid = prepare(train_who, train_porder, valid_who, valid_porder)
            
                train_set = lgb.Dataset(x, y)
                valid_set = lgb.Dataset(x_valid, y_valid, free_raw_data=False)

                watchlist = [valid_set]

                model = lgb.train(params, train_set=train_set, valid_sets=watchlist, num_boost_round=200, verbose_eval=50, early_stopping_rounds=100)

                for test_who in range(1, 8 + 1):
                    for test_porder in range(1, 3 + 1):

                        x_test = pfs[test_who - 1][test_porder - 1].event_primitive_df.drop(['hit_type'], axis=1)
                        y_true = pfs[test_who - 1][test_porder - 1].event_primitive_df['hit_type']

                        y_pred = model.predict(x_test, num_iteration=model.best_iteration)
                        predictions = []
                        for xx in y_pred:
                            predictions.append(np.argmax(xx))
                        y_test = pd.Series(data=predictions)

                        multierror = '%.4f' % get_multi_error(y_true, y_test)

    #                     train_str = '%d,%d' % (train_who, train_porder)
                        train_score = sfs[train_who - 1][train_porder - 1]

    #                     valid_str = '%d,%d' % (train_who, valid_porder)
                        valid_score = sfs[valid_who - 1][valid_porder - 1]

    #                     test_str = '%d,%d' % (test_who, test_porder)
                        test_score = sfs[test_who - 1][test_porder - 1]

                        result_df.loc[id_] = [train_who, train_porder, train_score, valid_who, valid_porder, valid_score, test_who, test_porder, test_score, multierror]

                        id_ += 1

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0
[100]	valid_0's multi_error: 0
Early stopping, best iteration is:
[8]	valid_0's multi_error: 0
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0886427
[100]	valid_0's multi_error: 0.0886427
Early stopping, best iteration is:
[33]	valid_0's multi_error: 0.0747922
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0858726
[100]	valid_0's multi_error: 0.0886427
Early stopping, best iteration is:
[5]	valid_0's multi_error: 0.0775623
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.202216
[100]	valid_0's multi_error: 0.204986
Early stopping, best iteration is:
[5]	valid_0's multi_error: 0.177285
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.185596
[100]	valid_0's multi_error: 0.210526
Early stopping, best iteration is:
[5]	v

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.162011
[100]	valid_0's multi_error: 0.162011
Early stopping, best iteration is:
[32]	valid_0's multi_error: 0.153631
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.138504
[100]	valid_0's multi_error: 0.141274
Early stopping, best iteration is:
[7]	valid_0's multi_error: 0.124654
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.135734
[100]	valid_0's multi_error: 0.146814
Early stopping, best iteration is:
[37]	valid_0's multi_error: 0.124654
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.119114
[100]	valid_0's multi_error: 0.105263
[150]	valid_0's multi_error: 0.113573
Early stopping, best iteration is:
[79]	valid_0's multi_error: 0.102493
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.168975
[100]	valid_0's multi_e

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0609418
[100]	valid_0's multi_error: 0.0609418
Early stopping, best iteration is:
[40]	valid_0's multi_error: 0.0554017
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0277008
[100]	valid_0's multi_error: 0.0277008
Early stopping, best iteration is:
[23]	valid_0's multi_error: 0.0277008
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0166205
[100]	valid_0's multi_error: 0.0166205
[150]	valid_0's multi_error: 0.0193906
Early stopping, best iteration is:
[55]	valid_0's multi_error: 0.0138504
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0249307
[100]	valid_0's multi_error: 0.0221607
Early stopping, best iteration is:
[18]	valid_0's multi_error: 0.0193906
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.182825
[100]	val

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.31856
[100]	valid_0's multi_error: 0.315789
Early stopping, best iteration is:
[4]	valid_0's multi_error: 0.260388
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0886427
[100]	valid_0's multi_error: 0.0886427
Early stopping, best iteration is:
[13]	valid_0's multi_error: 0.0803324
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.127424
[100]	valid_0's multi_error: 0.124654
Early stopping, best iteration is:
[28]	valid_0's multi_error: 0.119114
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.2
[100]	valid_0's multi_error: 0.2
Early stopping, best iteration is:
[16]	valid_0's multi_error: 0.177778
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.193906
[100]	valid_0's multi_error: 0.199446
Early stopping, best iteration 

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.207756
[100]	valid_0's multi_error: 0.207756
Early stopping, best iteration is:
[16]	valid_0's multi_error: 0.149584
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.171745
[100]	valid_0's multi_error: 0.166205
Early stopping, best iteration is:
[17]	valid_0's multi_error: 0.135734
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.138504
[100]	valid_0's multi_error: 0.144044
Early stopping, best iteration is:
[17]	valid_0's multi_error: 0.113573
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.159218
[100]	valid_0's multi_error: 0.156425
Early stopping, best iteration is:
[25]	valid_0's multi_error: 0.148045
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.135734
[100]	valid_0's multi_error: 0.135734
[150]	valid_0's multi_

...

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0720222
[100]	valid_0's multi_error: 0.0858726
Early stopping, best iteration is:
[43]	valid_0's multi_error: 0.0692521
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0858726
[100]	valid_0's multi_error: 0.0886427
Early stopping, best iteration is:
[39]	valid_0's multi_error: 0.0803324
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.360111
[100]	valid_0's multi_error: 0.351801
Early stopping, best iteration is:
[1]	valid_0's multi_error: 0.301939
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.304709
[100]	valid_0's multi_error: 0.293629
[150]	valid_0's multi_error: 0.290859
[200]	valid_0's multi_error: 0.296399
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.34626
[100]	valid_0's multi_error: 0.33795
Early stopping,

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.132964
[100]	valid_0's multi_error: 0.141274
Early stopping, best iteration is:
[23]	valid_0's multi_error: 0.116343
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.213889
[100]	valid_0's multi_error: 0.211111
Early stopping, best iteration is:
[1]	valid_0's multi_error: 0.188889
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.108033
[100]	valid_0's multi_error: 0.108033
Early stopping, best iteration is:
[28]	valid_0's multi_error: 0.0941828
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.124654
[100]	valid_0's multi_error: 0.119114
Early stopping, best iteration is:
[18]	valid_0's multi_error: 0.108033
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.099723
[100]	valid_0's multi_error: 0.105263
Early stopping, best i

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0498615
[100]	valid_0's multi_error: 0.0526316
Early stopping, best iteration is:
[23]	valid_0's multi_error: 0.0470914
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.066482
[100]	valid_0's multi_error: 0.0637119
Early stopping, best iteration is:
[26]	valid_0's multi_error: 0.0498615
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.203911
[100]	valid_0's multi_error: 0.203911
Early stopping, best iteration is:
[12]	valid_0's multi_error: 0.189944
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.152355
[100]	valid_0's multi_error: 0.171745
Early stopping, best iteration is:
[27]	valid_0's multi_error: 0.146814
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.132964
[100]	valid_0's multi_error: 0.152355
Early stopping, b

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0637119
[100]	valid_0's multi_error: 0.066482
Early stopping, best iteration is:
[5]	valid_0's multi_error: 0.0609418
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0747922
[100]	valid_0's multi_error: 0.0720222
Early stopping, best iteration is:
[28]	valid_0's multi_error: 0.0581717
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0498615
[100]	valid_0's multi_error: 0.0498615
Early stopping, best iteration is:
[15]	valid_0's multi_error: 0.0360111
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0277008
[100]	valid_0's multi_error: 0.0249307
Early stopping, best iteration is:
[9]	valid_0's multi_error: 0.0166205
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0360111
[100]	valid_0's multi_error: 0.0387812
Early stopp

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.163435
[100]	valid_0's multi_error: 0.152355
[150]	valid_0's multi_error: 0.149584
[200]	valid_0's multi_error: 0.149584
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.224377
[100]	valid_0's multi_error: 0.218837
Early stopping, best iteration is:
[21]	valid_0's multi_error: 0.207756
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.0554017
[100]	valid_0's multi_error: 0.0554017
Early stopping, best iteration is:
[35]	valid_0's multi_error: 0.0498615
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0
[100]	valid_0's multi_error: 0
Early stopping, best iteration is:
[8]	valid_0's multi_error: 0
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_error: 0.163889
[100]	valid_0's multi_error: 0.161111
Early stopping, best iteration is:
[1

In [14]:
result_df.to_csv('CSV/song1_result.csv', index=False)